# Implementing Backpropagation

### Introduction

In building and executing the hypothesis function, we were able to see we can build layers of a neural network, and connect them to make predictions.  But we would still like better understand how these these layers can make predictions that match our training data.  In other words, if we better understand the hypothesis function of a neural network, what about the training procedure of a neural network. 

Remember that a neural network has so far looked like the following.

`X -> lin1 -> nonlinear -> lin2 -> nonlinear -> predict_y -> mse`

### Implementing the training

Now remember that when we train a neural network, the question is, how do we alter the parameters in each of these layers so that they reduce the cost function.  

We should most alter the parameters that have causes the largest change in the cost function.  Or in other words, we would like to calculate the derivative with respect to each parameter that we can alter.  That is, we would like to calculate the gradient.

### The impact of the second linear layer

Currently, our neural network takes the following form.

`X -> lin1 -> nonlinear -> lin2 -> nonlinear -> predict_y -> mse`

And we can build these two layers linear layers through two different weight matrix.  Our task, when we train, is to find the changes to our weight matrices that will ultimately lead two predictions that minimize our mean squared error.  Now what makes this tricky is that the effect of our weight matrix, lin2, is indirect.  That is, the change in the weight matrix changes the output of the nonlinear component, which changes the predictions of y, which changes the mean squared error.

`lin2 -> nonlinear -> predict_y -> mse`

This is why we must apply the chain rule.  Applying the chain rule tells us that:

$$\frac{\delta C}{\delta L2_\theta} = \frac{\delta L2}{\delta L2_\theta}* \frac{\delta R}{\delta R_\theta} * \frac{\delta MSE}{\delta \hat{y}}$$

In other words, the change in the cost function as we change the weights in our second layer equals the change in the output of second layer, multiplied by the change in our non-linear function multiplied by the change in our cost function.  So now it's our task to calculate each of these components.

### Starting from the end

So if we just consider the last layer, the question would be how does altering the output `predict_y` decrease the cost of the function.  

`predict_y -> mse`

Calculating this follows the rules of our rules of derivatives.

$MSE(\theta) = (y - \hat{y})^2$

$\frac{\delta C}{\delta \hat{y}} = 2(y - \hat{y})$

In [ ]:
def mse_grad(pred, target):
    # grad of loss wrt previous layer (predictions)     
    return 2. *(pred.squeeze() - target).unsqueeze(-1) / pred.shape[0]

### Moving backwards

$relu_\theta -> predict\_y -> mse$

So now the question what is the impact of when the output of the relu function changes.

`mse(predict_y(relu(...)))`

$\frac{\delta C}{\delta R_\theta} = \frac{\delta R}{\delta R_\theta} * \frac{\delta C}{\delta \hat{y}}$

In [ ]:
def relu_grad(l_previous, grad_mse):
    #  so change will be proportional * grad_mse   
    return (l_previous>0).float() * grad_mse

* So notice that in non-linear gradient there is actually nothing to modify
* It's just to see the impact of the previous layer (So maybe should start at L2)

### Keep going backwards

$ lin\_2_\theta -> relu -> predict\_y -> mse$

`mse(predict_y(relu(lin_2())))`

$x \cdot W + b = \begin{bmatrix}
 -6& 9 &3  &-10
\end{bmatrix} \cdot \begin{bmatrix}
3 &4 \\ 
 2&5 \\ 
 10& 6\\ 
6 & 1
\end{bmatrix} + \begin{bmatrix}
10 \\ 
 2 \\ 
\end{bmatrix} = \begin{bmatrix} -20 \\ 31 \end{bmatrix}$

In [ ]:
def execute_linear(X, W, b):
    return (X@W) + b

So there are three components that can change the output.  
* the input value of X (here the previous layer)
* the weights of the matrix W 
* or the bias vector b.

So our critical task is to calculate the impact when of changing either of these parameters.  Once again we do this by calculating our gradient.

Calculating our gradient we have: 

$\frac{\delta C}{\delta L2_\theta} = \frac{\delta L2}{\delta L2_\theta} * \frac{\delta R}{\delta R_\theta} * \frac{\delta C}{\delta \hat{y}}$

Now if we we think back to the execution of the linear 

In [ ]:
def lin_grad(X, prev_grad, w, b):
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

### Linear Layer

* Gradient of a matrix product is the matrix product with the transpose

In [ ]:
def forward_and_backward(inp, targ):
    l1 = inp @w1 + b 
    l2 = relu(l1)
    output = l2 @ w2 + b2
    
    # backward pass
    mse_grad(output, targ)

    lin_grad(l2, out, w2, b2)
    relu_grad(l1, )
    lin_grad(inp, l1, w1, b1)

So $\frac{\delta z}{\delta w} = a^{L - 1}$

$\frac{\delta A^L}{\delta z} = \sigma'(z)$

$\frac{\delta C}{\delta A^L} = 2(y - a^L)$

or 

$ \frac{\delta C}{\delta w} = a^{L - 1}*\sigma'(z)*2(y - a^L)$

### Training loop

Now that we have implemented calculating the gradient for each layer, the next step is to progressively update our weight matrices by our gradient.

In [ ]:
def get_batches():
    batches = []
    # n -1 is size of the training set
    # maybe there is a way to just partition the training set, the iterate through     
    for i in range((n-1)//bs + 1):
        start_i = i*bs
        end_i = start_i+bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        batches.append((xb, yb))
    return batches

In [ ]:
lr = .5 # Set a learning rate.
epochs = 1 # how many cycles to train for

In [ ]:
for epoch in range(epochs):
    for xb, yb in get_batches():
        hypothesis = model(xb)
        loss = loss_func(hypothesis, yb)
        
        # this is the mse.loss, really gradient
        # because needs to set the gradient for the chain rule         
        loss.backward()
        # When call loss.backward(), this recalculates all of the gradients for each layer
        with torch.no_grad():
            for l in model.layers:
                if has_attr(l, 'weight'):
                    # go through each layer
                    # find the gradient (but does weight.grad find it?)
                    # and decrease by learning rate                     
                    l.weight -= l.weight.grad * lr
                    # this gradient is how much each weight it multiplied by, 
                    # so decrease each weight by the amount it is impacted                          
                    l.bias -= l.bias.grad * lr
                    
                    # zero the gradients because 
                    # loss function recalculates the gradients                     
                    l.weight.grad.zero_()
                    l.bias.grad.zero_()

* dl1/lesson2-sgd SGD - 

* 46:15 https://www.youtube.com/watch?v=AcA8HAYh7IE&t=888s

[fast_ai notebook](https://github.com/fastai/course-v3/blob/master/nbs/dl2/03_minibatch_training.ipynb)

[michael nielsen](http://neuralnetworksanddeeplearning.com/chap4.html)

* Look at building a neural network from scratch - especially diy with python